In [4]:
import os
import numpy as np
import json
from pathlib import Path
from keras.models import Model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense, AveragePooling2D,Conv2D
)
from keras.layers import ( Input, Activation, Dense, Flatten )
#from keras.layers.convolutional import ( Conv2D, MaxPooling2D , AveragePooling2D) //DMR se utilizo tensorflow.keras.layers
from keras.layers.merge import add
#from keras.layers.normalization import BatchNormalization //DMR se utilizo tensorflow.keras.layers
from keras.regularizers import l2
from keras import backend as K

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping



In [5]:
path = os.path.abspath(os.getcwd())+'\\resultados\\'

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

train_data = np.load(path+'train_data.npy')
train_labels = np.load(path+'train_labels.npy')
test_data = np.load(path+'test_data.npy')
test_labels = np.load(path+'test_labels.npy')
labels = np.load(path+'labels.npy')

# restore np.load for future normal usage
np.load = np_load_old

In [6]:
print( "train data shape: ",  train_data.shape )
print( "train label shape: ", train_labels.shape )
print( "test data shape: ",   test_data.shape )
print( "test_labels.shape: ", test_labels.shape )

train data shape:  (100000, 64, 64, 3)
train label shape:  (100000, 200)
test data shape:  (10000, 64, 64, 3)
test_labels.shape:  (10000, 200)


In [7]:
def shuffle_data(train_data, train_labels ):
    size = len(train_data)
    train_idx = np.arange(size)
    np.random.shuffle(train_idx)

    return train_data[train_idx], train_labels[train_idx]
  
train_data, train_labels = shuffle_data(train_data, train_labels)

In [8]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger('resnet50_tiny_ImageNet.csv')

batch_size = 100
nb_classes = 200
nb_epoch = 10

# input image dimensions
img_rows, img_cols = 64, 64
# The images are RGB
img_channels = 3

input_shape = (img_rows, img_cols, img_channels)

# The data, shuffled and split between train and test sets:
X_train = train_data
Y_train = train_labels
X_test = test_data
Y_test = test_labels
del train_data
del train_labels
del test_data
del test_labels
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# subtract mean and normalize
mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_test -= mean_image
X_train /= 128.
X_test /= 128.


In [9]:
# Se utilia un modelo de KERAS el resnet50
resnet50v2 = keras.applications.ResNet50V2(include_top=True, classes=nb_classes, input_shape=input_shape, weights=None)
#resnet50v2.summary()

resnet50v2.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['acc'])


In [10]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
          featurewise_center=False,           # set input mean to 0 over the dataset
          samplewise_center=False,            # set each sample mean to 0
          featurewise_std_normalization=False,# divide inputs by std of the dataset
          samplewise_std_normalization=False, # divide each input by its std
          zca_whitening=False,                # apply ZCA whitening
          rotation_range=0,                   # randomly rotate images in the range (degrees, 0 to 180)
          width_shift_range=0.1,              # randomly shift images horizontally (fraction of total width)
          height_shift_range=0.1,             # randomly shift images vertically (fraction of total height)
          horizontal_flip=True,               # randomly flip images
          vertical_flip=False )               # randomly flip images

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit( X_train )

Using real-time data augmentation.


In [11]:
nb_epoch = 10 # epoch 1-10
# Fit the model on the batches generated by datagen.flow().
resnet50v2.fit_generator( datagen.flow(X_train, Y_train, batch_size=batch_size),
                     steps_per_epoch=X_train.shape[0] // batch_size,
                     validation_data=(X_test, Y_test),
                     epochs=nb_epoch, verbose=1,
                     callbacks=[lr_reducer, early_stopper, csv_logger] )
                     
# El modelo es entrenado con el dataset cifar10
with open("history_resnet50v2.json", "w") as ff:
    json.dump(resnet50v2.history, ff)

C:\Users\DAVID\AppData\Local\Temp/ipykernel_22288/1204747456.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  resnet50v2.fit_generator( datagen.flow(X_train, Y_train, batch_size=batch_size),


Epoch 1/10
1000/1000 [==============================] - 4784s 5s/step - loss: 4.7127 - acc: 0.0604 - val_loss: 4.4708 - val_acc: 0.0809 - lr: 0.0010
Epoch 2/10
1000/1000 [==============================] - 4715s 5s/step - loss: 3.9143 - acc: 0.1473 - val_loss: 4.6278 - val_acc: 0.0928 - lr: 0.0010
Epoch 3/10
1000/1000 [==============================] - 4984s 5s/step - loss: 3.6063 - acc: 0.1962 - val_loss: 3.5405 - val_acc: 0.2055 - lr: 0.0010
Epoch 4/10
1000/1000 [==============================] - 4689s 5s/step - loss: 3.2834 - acc: 0.2482 - val_loss: 3.2266 - val_acc: 0.2640 - lr: 0.0010
Epoch 5/10
1000/1000 [==============================] - 4574s 5s/step - loss: 3.0907 - acc: 0.2836 - val_loss: 3.1094 - val_acc: 0.2841 - lr: 0.0010
Epoch 6/10
1000/1000 [==============================] - 4817s 5s/step - loss: 2.8924 - acc: 0.3183 - val_loss: 3.1032 - val_acc: 0.2869 - lr: 0.0010
Epoch 7/10
1000/1000 [==============================] - 4829s 5s/step - loss: 2.7220 - acc: 0.3513 - val_l

TypeError: Object of type History is not JSON serializable

In [12]:
resnet50v2.save('models/resnet50v2')


INFO:tensorflow:Assets written to: models/resnet50v2\assets


C:\Users\DAVID\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\DAVID\anaconda3\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [16]:

resnet50v2 = tf.keras.models.load_model('models/resnet50v2')

# Check its architecture
resnet50v2.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 70, 70, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 32, 32, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 34, 34, 64)   0           ['conv1_conv[0][0]']             
                                                                                         

In [17]:
resnet50v2.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 70, 70, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 32, 32, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 34, 34, 64)   0           ['conv1_conv[0][0]']             
                                                                                         